<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+GS+LNS
Merge the bibliographic datasets for 
* Web of Science, 
* Scielo 
* Scopus 
* Google Scholar
* Puntaje
* Lens
of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

## functions

In [1]:
import unidecode
def get_author_info(x):
    sep='; '
    authors=[{'WOS_author':x[0].split(sep)[0],'affiliation':[x[0].split(sep)[-1]],'i':0}]
    iau=1
    for y  in x:
        y2=y.replace('[','').replace('] ',sep).split(sep)
        for z in y2[:-1]:
            aulist=[ d.get('WOS_author') for d in authors]
            if z not in aulist:
                authors.append({'WOS_author':z,'affiliation':[y2[-1]],'i':iau})
                iau=iau+1
            else:
                if y2[-1] not in [ d.get('affiliation') for d in authors if d.get('WOS_author')==z][0]:
                    index_author=[ d.get('i') for d in authors if d.get('WOS_author')==z][0]
                    authors[index_author]['affiliation'].append(y2[-1])
    return authors

def dictionary_list_add_columns(df,df_dl,df_dl_key,df_dl_i,df_columns):
    '''
    For a
     df: Pandas DataFrame 
    with a:
     df_dl: column of list of dictionaries, with
     df_dl_key: dictionary key: e.g x=[{df_dl_key:1},{df_dl_key:2}]
    for the element df_dl_i of the list:
    Update the dictionary with:
        df_dl_key==x[df_dl_i][df_dl_key]
    with the dictionaries { df_columns[i]: df_columns[i].values }
    '''
    dff=df.copy()
    for key in df_columns:
        tmp=dff[df_dl].combine(dff[key],
                func=lambda x,y: y if pd.isna(y) 
                                   else 
                                     [z.update({key:y}) 
                                     if z.get(df_dl_key)==x[df_dl_i][df_dl_key] 
                                     else z 
                                 for z in x  ] )
    return dff

def split_full_names(y,full_name='full_name'):
    yy=y.get(full_name).title()
    lfn=len(y[full_name].split())
    aps=0
    d={ 'PRIMER APELLIDO':yy.split()[aps] }
    aps=aps+1
    if lfn>=4:
        names=-2
        if len( d['PRIMER APELLIDO'] )<=3:
            d['PRIMER APELLIDO']=d['PRIMER APELLIDO']+' '+yy.split()[aps]
            aps=aps+1
            d.update(  {'SEGUNDO APELLIDO':yy.split()[aps]} )
            names=names+1
            
        d.update({'SEGUNDO APELLIDO':yy.split()[aps]})
        aps=aps+1
        if len( d['SEGUNDO APELLIDO'] )<=3:
            d['SEGUNDO APELLIDO']=d['SEGUNDO APELLIDO']+' '+yy.split()[aps]
            if names==-2:
                names=names+1
    elif lfn>=3:        
        d.update({'SEGUNDO APELLIDO':yy.split()[aps]})
        names=-1
    else: #Colombian interpretation (TODO: Includes Brazilian interpretation)    
        names=-1
    d.update({'NOMBRES':' '.join( yy.split()[names:]),
              'INICIALES':' '.join( [z[0]+'.' for z in yy.split()[names:]] ),
              })
    return d

In [2]:
import wosplus as wp
import pandas as pd

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
UDEA_WOS.xlsx       = 1px2IcrjCrkyu7t78Q7PAE5nzV_yuPt9t
UDEA_SCI.xlsx       = 1pWMY5P72j0Ca6D-cm7dn7Q4TBGTs4PWV
UDEA_SCP.xlsx       = 1ulCsFHzDiTmuL9TH8F58ulh0u8Z2ylKh
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
UDEA_WOS_SCI_SCP.json=1RTDCh5pl0vapjJT_e9ZwadHPGBKGGv6Y
UDEA_WOS_SCI_SCP.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs

Overwriting drive.cfg


##  Load data bases

In [4]:
drive_files=wp.wosplus('drive.cfg')

In [ ]:
UDEA=drive_files.read_drive_json('UDEA_WOS_SCI_SCP.json')

In [ ]:
UDEA.Tipo.unique()

In [ ]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA[ UDEA.Tipo==t].shape[0] ) )

## Extract  affiliation from C1

In [ ]:
affil='Univ Antioquia'
UDEA['authors_WOS']=UDEA.C1.apply(lambda x: x.split('\n') if x else x).apply(
    lambda x:   [y.replace('[','').replace('] ','; ') for y in x if y.find(affil)>-1 ] if x else x ).apply(
     lambda x: get_author_info(x) if x else x)

## load trained 

In [ ]:
SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')

In [ ]:
SIU.Tipo.unique()

In [ ]:
SIU=SIU[SIU.Tipo.str.contains('\+UDEA')].reset_index(drop=True)

In [ ]:
SIU.columns.values

TODO: def: Convert value string list into a list of dictionaries

In [ ]:
SIU['UDEA_authors']=SIU.UDEA_autores.str.split(';').apply(lambda x: [{'full_name':y} for y in x ])

### Merge with official researcher list

BUG: Only first author information added

In [ ]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [ ]:
AU_columns=list( AU.columns.values )

In [ ]:
AU['name_tmp']=(AU['PRIMER APELLIDO']+' '+AU['SEGUNDO APELLIDO']+' '+AU['NOMBRES']).str.lower().str.strip().apply( 
    unidecode.unidecode )

In [ ]:
import unidecode

In [ ]:
maxau=SIU['UDEA_authors'].apply(lambda x: [y.get('full_name') for y in x ]).apply(len).max()
SIUnew=pd.DataFrame()
kkn=SIU.copy()

In [ ]:
newcolumns=['name_tmp']+AU_columns
for i in range(maxau):
    print(i)
    kkn['name_tmp']=kkn['UDEA_authors'].apply(lambda x: [y.get('full_name') for y in x ]
                            ).str[i].apply( lambda x: unidecode.unidecode( x.lower().strip()) 
                                                      if not pd.isna(x) else x)
    if not kkn[~kkn['name_tmp'].isna()].empty:
        kkn=kkn.merge(AU[newcolumns],on='name_tmp',how='left').reset_index(drop=True)
        kkn=dictionary_list_add_columns(kkn,'UDEA_authors','full_name',i,AU_columns)
        kkn=kkn.drop(newcolumns,axis='columns')

https://stackoverflow.com/a/29530601/2268280

BUG: Cells filled with nans

In [ ]:
kkn.shape,SIU.shape

for i in range(maxau):
    print(i)
    kkn['name_tmp']=kkn['UDEA_authors'].apply(lambda x: [y.get('full_name') for y in x ]
                            ).str[i].apply( lambda x: unidecode.unidecode( x.lower().strip()) 
                                                      if not pd.isna(x) else x)

    newcolumns=['name_tmp']+AU_columns
    if not kkn[~kkn['name_tmp'].isna()].empty:
        kk=kkn.merge(AU[newcolumns],on='name_tmp',how='left')
        kky=kk[~kk['NOMBRE COMPLETO'].isna()].reset_index(drop=True)
        kkn=kk[ kk['NOMBRE COMPLETO'].isna()].reset_index(drop=True).drop(
               newcolumns,axis='columns')
        kky=dictionary_list_add_columns(kky,'UDEA_authors','full_name',i,AU_columns)
        
        SIUnew=SIUnew.append( kky ).reset_index(drop=True)
        #update str[i]
    else:
        SIUnew=SIUnew.append( kkn, sort=False ).reset_index(drop=True)
        break

In [ ]:
SIUnew.UDEA_authors.values

In [ ]:
newcolumns

In [ ]:
SIU=kkn.copy()

In [ ]:
AU[ AU['NOMBRE COMPLETO'].str.contains('Oscar Alberto Zapata Noreña') ]

In [ ]:
# Creates mask Search key in a list of dictionay
# First apply convert null values to string
# Second apply: implement a mask
def search_key_in_list_of_dictionaries(df,column,key,pattern):
    return df[column].apply(lambda x: 
                [ '' if pd.isnull( y.get(key)) else y for y in x ]  ).apply(
                            lambda x: 
                [ True if y.get(key).find(pattern)>-1 else False for y in x  ][0]  )

In [ ]:
SIU[ search_key_in_list_of_dictionaries(SIU,'UDEA_authors','full_name','ZAPATA') ].UDEA_authors.loc[241]

In [ ]:
SIU[ search_key_in_list_of_dictionaries(SIU,'UDEA_authors','full_name','PONCE') ].UDEA_authors.loc[147]

In [ ]:
SIU[ SIU.UDEA_authors.apply(lambda x: [ '' if pd.isnull( y.get('full_name')) else y for y in x ]  ).apply(
    lambda x: [ True if y.get('full_name').find('ZAPATA')>-1 else False for y in x  ][0]  )].UDEA_authors.loc[148]

In [ ]:

SIU[ SIU.UDEA_authors.apply(lambda x: [ '' if pd.isnull( y.get('full_name')) else y for y in x ]  ).apply(
    lambda x: [ True if y.get('full_name').find('PONCE')>-1 else False for y in x  ][0]  )].UDEA_authors.loc[147]

TODO: Falta añadir datos a nombres que faltan

### Merge with trained data set

In [ ]:
SIUDI=SIU[~SIU.DI.isna()].drop_duplicates('DI').reset_index(drop=True)
SIUTI=SIU[ SIU.DI.isna()].drop_duplicates('TI').reset_index(drop=True)
SIUTI=SIUTI[SIUTI!=''].reset_index(drop=True)
SIUTI=SIUTI[~SIUTI.TI.isnull()].reset_index(drop=True)
SIUTI=SIUTI[ SIUTI.TI.apply(len)>20 ].reset_index(drop=True)

In [ ]:
udea_columns=[       'UDEA_autores',
       'UDEA_año realiz', 'UDEA_doi', 'UDEA_fecha aplicación',
       'UDEA_idioma', 'UDEA_item adic', 'UDEA_material', 'UDEA_nombre',
       'UDEA_nombre revista o premio', 'UDEA_nro autores', 'UDEA_país',
       'UDEA_procodigo', 'UDEA_ptos', 'UDEA_simple_doi', 'UDEA_título',
       'UDEA_valor item','UDEA_authors']

In [ ]:
UDEADI=UDEA[UDEA.DI!=''].drop_duplicates('DI').reset_index(drop=True)
UDEATI=UDEA[UDEA.DI==''].drop_duplicates('TI').reset_index(drop=True)

In [ ]:
UDEA_mergeDI=UDEADI.merge( SIUDI[ ['DI']+udea_columns ],on='DI',how='left' )

In [ ]:
UDEADI.shape,UDEA_mergeDI.shape

In [ ]:
UDEA_PTJ=UDEA_mergeDI[~UDEA_mergeDI.UDEA_autores.isna()].reset_index(drop=True)
UDEA_PTJ_NOT=UDEA_mergeDI[UDEA_mergeDI.UDEA_autores.isna()].reset_index(drop=True)

In [ ]:
UDEATI['tmptitle']=UDEATI.TI.str.strip()
SIUTI['tmptitle']=SIUTI.TI.str.strip()

In [ ]:
kk=UDEATI.merge( SIUTI[ ['tmptitle']+udea_columns ],on='tmptitle',how='left' ).drop('tmptitle',axis='columns')

In [ ]:
UDEA_PTJ=UDEA_PTJ.append( kk[ ~kk.UDEA_autores.isna() ] ).reset_index(drop=True)
UDEA_PTJ_NOT=UDEA_PTJ_NOT.append( kk[ kk.UDEA_autores.isna() ] ).reset_index(drop=True)

In [ ]:
UDEA_PTJ.shape[0]+UDEA_PTJ_NOT.shape[0],UDEA.shape

In [ ]:
UDEA_PTJ.shape,UDEA_PTJ_NOT.shape

In [ ]:
UDEA_PTJ.loc[[0,1]]

## Extrapolates to UDEA_PTJ_NOT

### Defina missing key variation in UDEA_authors

In [ ]:
UDEAnew=UDEA_PTJ.append(
    UDEA_PTJ_NOT).reset_index(
    drop=True)

In [ ]:
tmp=UDEAnew['UDEA_authors'].apply(lambda x: [y.update( 
                split_full_names(y,full_name='full_name')  ) if not pd.isnull(
                y.get('full_name')) else y for y in x] 
                                   if type(x)==list 
                                   else x)

In [ ]:
UDEAnew.UDEA_authors.loc[4]

### merge separated names of UDEA_PTJ.authors_WOS with UDEA_PTJ.authors data 

In [ ]:
#x,y=UDEA_PTJ['authors_WOS'].combine( UDEA_PTJ['UDEA_authors'], func=lambda x,y:(x,y)).loc[0]

In [ ]:
x

In [ ]:
y

In [ ]:
def wos_names_list(dy ,y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES']   ):
    last_name= unidecode.unidecode( dy[y_keys[0]]   )
    first_names=unidecode.unidecode( dy[y_keys[1]]  )
    intials=unidecode.unidecode( dy[y_keys[2]]  )
    wos_names=[           last_name+', '+first_names]
    if len( first_names.split())>1:
        wos_names=wos_names+[ last_name+', '+first_names.split()[0] ]
    wos_names=wos_names+[ last_name+', '+intials]
    if len( intials.split())>1:
        wos_names=wos_names+[ last_name+', '+intials.split()[0]]
    if len(intials.split())==2:
        wos_names=wos_names+[
              last_name+', '+first_names.split()[0]+' '+intials.split()[-1] ]
    return wos_names
    
def combinewos(x,y,x_keys=['WOS_author','affiliation'],
                   y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES'],
                   xy_keys=['wos_author','wos_affiliation']):
    for dx in x:
        wos_name=unidecode.unidecode( dx[ x_keys[0] ] )
        wos_affiliation= dx[x_keys[1]]
        for i in range( len(y) ):
            if wos_name in wos_names_list(y[i] ,y_keys):
                y[i][  xy_keys[0] ]=[ wos_name ]
                y[i][  xy_keys[1] ]=wos_affiliation 
    return y

In [ ]:
combinewos(x,y)

In [205]:
kk=UDEA_PTJ['authors_WOS'].combine( UDEA_PTJ['UDEA_authors'], func=combinewos )

In [207]:
kk[1]

[{'CÉDULA': 1017126043.0,
  'DEPARTAMENTO': 'Departamento de Siquiatría y Ciencias del Comportamiento Hum',
  'FACULTAD': 'Facultad de Medicina',
  'GRUPO': 'Grupo de Investigación en Psiquiatría GIPSI',
  'INICIALES': 'C. D.',
  'NOMBRE COMPLETO': 'Cristian David Vargas Upegui',
  'NOMBRES': 'Cristian David',
  'PRIMER APELLIDO': 'Vargas',
  'SEGUNDO APELLIDO': 'Upegui',
  'full_name': 'VARGAS UPEGUI CRISTIAN DAVID',
  'wos_affiliation': [],
  'wos_name': []},
 {'CÉDULA': 71696926.0,
  'DEPARTAMENTO': 'Departamento de Siquiatría y Ciencias del Comportamiento Hum',
  'FACULTAD': 'Facultad de Medicina',
  'GRUPO': 'Grupo de Investigación en Psiquiatría GIPSI',
  'INICIALES': 'C. A.',
  'NOMBRE COMPLETO': 'Carlos Alberto Lopez Jaramillo',
  'NOMBRES': 'Carlos Alberto',
  'PRIMER APELLIDO': 'Lopez',
  'SEGUNDO APELLIDO': 'Jaramillo',
  'full_name': 'LOPEZ JARAMILLO CARLOS ALBERTO',
  'wos_affiliation': [],
  'wos_name': []}]

In [209]:
y[iy]['wos_name']=y[iy]['wos_name']+[wos_name]
y[iy]['wos_affiliation']=y[iy]['wos_affiliation']+wos_affiliation
y[iy] 

IndexError: list index out of range

In [208]:
print 1

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-208-c94594b6b28f>, line 1)

In [50]:
UDEA_PTJ[ UDEA_PTJ['authors_WOS'].str[0].apply(lambda x: {} if pd.isnull(x) else x).apply(
                       lambda x: x.get('WOS_author') if x else '').str.contains(
        'Restrepo, D') ]['authors_WOS']#.reset_index(drop=True).loc[:3].values

786     [{'affiliation': ['Univ Antioquia, Inst Fis, C...
852     [{'affiliation': ['Univ Antioquia, Inst Fis, C...
1061    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
1600    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
1656    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
1729    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
1931    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2055    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2088    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2421    [{'affiliation': ['Univ Antioquia, Inst Fis, B...
2427    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2618    [{'affiliation': ['Univ Antioquia, Medellin 12...
2663    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2674    [{'affiliation': ['Univ Antioquia, Medellin, C...
2687    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2750    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2787    [{'affiliation': ['Univ Antioquia, Inst Fis, M...
2837    [{'aff

In [1443]:
UDEA_PTJ[UDEA_PTJ['authors_WOS'].str[0].fillna('').str.contains('Restrepo, D')]

ValueError: cannot index with vector containing NA / NaN values

In [ ]:
#def update_WOS_author(x,y):
#return UDEA_authors( 'WOS_author':[list of possible names]...,'WOS_affiliation')    

Combines the two list into a single one in  UDEA_authors

In [1216]:
au={'last_names':'Dover, Robert V. H.'.split(', '),'first_names':'Dover, Robert V. H.'.split(', ')[-1],
    'initials':[ y  for y in au['first_names'].split() if len(y)<=2 ] }

In [1215]:
au

{'first_names': 'Robert V. H.',
 'initials': ['V.', 'H.'],
 'last_names': ['Dover', 'Robert V. H.']}

In [1206]:
def combine_wos_udea_authors(df):
    '''
    add 'WOS_author and WOS_affiliation to the respective UDEA_authors dictionary'
    '''
    pass

In [1137]:
UDEA_PTJ[['authors_WOS','UDEA_authors']].loc[12].values

array([list([{'WOS_author': 'Monsalve, Juan', 'affiliation': ['Univ Antioquia Medellin, Inst Matemat, Medellin, Colombia.'], 'i': 0}, {'WOS_author': 'Rada, Juan', 'affiliation': ['Univ Antioquia Medellin, Inst Matemat, Medellin, Colombia.'], 'i': 1}]),
       list([{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0, 'GRUPO': 'Álgebra U de A', 'SEGUNDO APELLIDO': 'Rincon', 'FACULTAD': 'Facultad de Ciencias Exactas y Naturales', 'NOMBRE COMPLETO': 'Juan Pablo Rada Rincon', 'PRIMER APELLIDO': 'Rada', 'full_name': 'RADA RINCON JUAN PABLO', 'DEPARTAMENTO': 'Departamento de Matemáticas'}])],
      dtype=object)

check

Full query of auhor_WOS in UDEA_authors:

In [1203]:
primer_apellido='Rada'
primer_nombre='Juan'
UDEA_PTJ[ UDEA_PTJ.UDEA_authors.apply(lambda x: [y.get('PRIMER APELLIDO')+' '
                                                 +y.get('SEGUNDO APELLIDO')
                                                 +', '+y.get('NOMBRES') if not pd.isnull(y.get('SEGUNDO APELLIDO')) else '' for y in x ]
                                     ).str[0].fillna('').str.contains('^%s[A-Za-z\ ]*, %s' %(
                                                             primer_apellido,primer_nombre)) ]

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,UDEA_nombre,UDEA_nombre revista o premio,UDEA_nro autores,UDEA_país,UDEA_procodigo,UDEA_ptos,UDEA_simple_doi,UDEA_título,UDEA_valor item,UDEA_authors
12,If D is a digraph with n vertices then the ene...,"Monsalve, Juan\nRada, Juan\n",,"Monsalve, J\nRada, J\n",,,,,124\n,"[Monsalve, Juan; Rada, Juan] Univ Antioquia Me...",...,RADA RINCON JUAN PABLO,APPLIED MATHEMATICS AND COMPUTATION,2.0,74.0,0.0,0.0,0.0,BICYCLIC DIGRAPHS WITH MAXIMAL ENERGY.,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
123,,,,"Cruz R., Rada J.",,,,,,,...,RADA RINCON JUAN PABLO,APPLIED MATHEMATICAL SCIENCES,2.0,28.0,0.0,0.0,0.0,EXTREMAL VALUES OF VDB TOPOLOGICAL INDICES OVE...,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
189,,,,Rada J.,,,,,,,...,RADA RINCON JUAN PABLO,APPLIED MATHEMATICAL SCIENCES,1.0,28.0,0.0,0.0,0.0,THE LINEAR CHAIN AS AN EXTREMAL VALUE OF VDB T...,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
604,The trace norm of the digraph D is defined as ...,"Agudelo, N.\nde la Pena, J. A.\nRada, J.\n",,"Agudelo, N\nde la Pena, JA\nRada, J\n",,,,,261\n,"[Agudelo, N.; Rada, J.] Univ Antioquia, Inst M...",...,RADA RINCON JUAN PABLO,LINEAR ALGEBRA AND ITS APPLICATIONS,3.0,74.0,54940.0,12.0,0.0,EXTREMAL VALUES OF THE TRACE NORM OVER ORIENTE...,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
1139,The energy of a graph G is defined as E(G) = S...,"Agudelo, Natalia\nRada, Juan\n",,"Agudelo, N\nRada, J\n",,,,,156\n,"[Agudelo, Natalia; Rada, Juan] Univ Antioquia,...",...,RADA RINCON JUAN PABLO,LINEAR ALGEBRA AND ITS APPLICATIONS,2.0,74.0,0.0,0.0,0.0,LOWER BOUNDS OF NIKIFOROV'S ENERGY OVER DIGRAPHS.,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
1343,Let G be a graph with n vertices. A vertex-deg...,"Berrocal, Lilia\nOlivieri, Aurora\nRada, Juan\n",,"Berrocal, L\nOlivieri, A\nRada, J\n",,,,,176\n,"[Berrocal, Lilia; Olivieri, Aurora] Univ Simon...",...,RADA RINCON JUAN PABLO,APPLIED MATHEMATICS AND COMPUTATION,3.0,74.0,0.0,0.0,0.0,EXTREMAL VALUES OF VERTEX-DEGREE-BASED TOPOLOG...,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
1534,The Wiener index of a connected graph G is the...,"Gutman, Ivan\nCruz, Roberto\nRada, Juan\n",,"Gutman, I\nCruz, R\nRada, J\n",,,,,247\n,"[Gutman, Ivan] Univ Kragujevac, Fac Sci, Kragu...",...,RADA RINCON JUAN PABLO,DISCRETE APPLIED MATHEMATICS,3.0,74.0,0.0,0.0,0.0,WIENER INDEX OF EULARIAN GRAPHS,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
3671,Let HSh denote the set of hexagonal systems wi...,"Cruz, Roberto\nDuque, Frank\nRada, Juan\n",,"Cruz, R\nDuque, F\nRada, J\n",,,,,707\n,"[Cruz, Roberto; Rada, Juan] Univ Antioquia, In...",...,RADA RINCON JUAN PABLO,MATCH-COMMUNICATIONS IN MATHEMATICAL AND IN CO...,3.0,74.0,55876.0,15.0,0.0,HEXAGONAL SYSTEMS WITH MINIMAL NUMBER OF INTELS.,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
3705,"Given a graph G with n vertices, a vertex-degr...","Rada, Juan\nCruz, Roberto\n",,"Rada, J\nCruz, R\n",,,,,603\n,"[Rada, Juan; Cruz, Roberto] Univ Antioquia, In...",...,CRUZ RODES ROBERTO,MATCH-COMMUNICATIONS IN MATHEMATICAL AND IN CO...,2.0,162.0,0.0,0.0,0.0,VERTEX-DEGREE-BASED TOPOLOGICAL INDICES OVER G...,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."
6941,We show that digraphs with exactly two eigenva...,"Olivieri, A.\nRada, J.\nRios, A. J.\n",,"Olivieri, A\nRada, J\nRios, AJ\n",,,,,89\n,"[Olivieri, A.; Rada, J.; Rios, A. J.] Univ Sim...",...,RADA RINCON JUAN PABLO,UTILITAS MATHEMATICA,3.0,34.0,0.0,0.0,0.0,DIGRAPHS WITH FEW EIGENVALUES,NaN,"[{'NOMBRES': 'Juan Pablo ', 'CÉDULA': 413218.0..."


# TMP

In [1119]:
UDEA[ UDEA['authors_WOS'].apply(lambda x: [d for d in  x if d.get('WOS_author')=='Velasquez, Jesus A.']
                                if x else x).apply(len)>0 ]['authors_WOS'].str[0].apply( 
    lambda x: x.get('affiliation')).values

array([list(['Univ Antioquia, Hosp Univ San Vicente Fdn, Medellin, Colombia.', 'Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.', 'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.']),
       list(['Univ Antioquia, Hosp Univ San Vicente Fdn, Medellin, Colombia.', 'Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.', 'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.']),
       list(['Univ Antioquia, Hosp Univ San Vicente Fdn, Medellin, Colombia.', 'Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.', 'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.'])],
      dtype=object)

In [1120]:
y2[-1],z

('Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.',
 'Tolosa, Jorge E.')

In [1121]:
[ d.get('affiliation') for d in authors if d.get('WOS_author')==z][0]

['Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.',
 'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.']

In [1122]:
y2[-1] not in [ d.get('affiliation') for d in authors if d.get('WOS_author')==z][0]

False

In [1125]:
authors[0]['affiliation'].append('kk')

In [1126]:
authors

[{'WOS_author': 'Velasquez, Jesus A.',
  'affiliation': ['Univ Antioquia, Hosp Univ San Vicente Fdn, Medellin, Colombia.',
   'Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.',
   'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.',
   'kk'],
  'i': 0},
 {'WOS_author': 'Tolosa, Jorge E.',
  'affiliation': ['Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.',
   'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.'],
  'i': 1}]

In [1127]:
affs=UDEA.C1.apply(lambda x: x.split('\n') if x else x).apply(
    lambda x:   [y.replace('[','').replace('] ','; ') for y in x if y.find(affil)>-1 ] if x else x )

In [1128]:
affsf=affs.apply(lambda x: [  y.split('; ') for y in x])

In [1129]:
affsf.apply( lambda x: [ y.append(x) for y in x]).loc[i]

[None]

In [349]:
import numpy as np

In [352]:
np.array(x).flatten()

array([list(['Velasquez, Jesus A.']),
       list(['Velasquez, Jesus A.', 'Tolosa, Jorge E.']),
       list(['Velasquez, Jesus A.', 'Tolosa, Jorge E.'])], dtype=object)

In [326]:
affs

0        [[Burwick, Richard M.; Tolosa, Jorge E.] Orego...
1        [[Burwick, Richard M.; Rincon, Monica; Tolosa,...
10       [[Castaneda Gallego, Alba D.; Cardona Arias, J...
100      [[Herrera Mejia, Julian; Herrera Mejia, Julian...
1000     [[Mesa, C.; Giraldo, C. A.; Angulo, J.; Ruiz, ...
10000                                                   []
10001                                                   []
10002                                                   []
10003                                                   []
10004                                                   []
10005                                                   []
10006                                                   []
10007                                                   []
10008                                                   []
10009                                                   []
1001     [[Giraldo-Echeverri, C. A.; Taborda, N.; Ruiz,...
10010                                                   

In [306]:
i=120
i=1
affil='Univ Antioquia'
affs=UDEA.C1.str.split('\n').apply(
    lambda x:   [ [z,y.replace('[','').split('] ')[-1]
                  for z in y.replace('[','').split('] ')[0].split('; ')
                  ] for y in x if y.find(affil)>-1 ]if x else x).loc[i]

SyntaxError: invalid syntax (<ipython-input-306-8c7b399dee8b>, line 6)

In [303]:
{ x:'A' for x in [1,2]} 

{1: 'A', 2: 'A'}

In [304]:
affs

[{'Velasquez, Jesus A.': 'Univ Antioquia, Hosp Univ San Vicente Fdn, Medellin, Colombia.'},
 {'Tolosa, Jorge E.': 'Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.',
  'Velasquez, Jesus A.': 'Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.'},
 {'Tolosa, Jorge E.': 'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.',
  'Velasquez, Jesus A.': 'Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.'}]